In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load('wiki_w2v.model')

In [ ]:
pos=pd.read_csv('new_weibo_data_clean.csv',names=['idx','content'])
neg=pd.read_csv('new_weibo_data_clean0.csv',names=['idx','content'])

In [ ]:
pos_arr=[]
pos_la=[]
for line in pos.values:
    length=0
    vecs=np.zeros((1,300))
    newline=line[0].split(' ')
    for i in newline:
        try:
            vecs+=model[i].reshape((1,300))
            length+=1
        except KeyError:
            continue  
    pos_arr.append(vecs/length)
    pos_la.append(float(1))
pos_arr=np.array(pos_arr).reshape(-1,300)
pos_la=np.array(pos_la).reshape((-1,1))

neg_arr=[]
neg_la=[]
for line in neg.values:
    length=0
    vecs=np.zeros((1,300))
    newline=line[0].split(' ')
    for i in newline:
        try:
            vecs+=model[i].reshape((1,300))
            length+=1
        except KeyError:
            continue  
    neg_arr.append(vecs/length)
    neg_la.append(float(0))
neg_arr=np.array(neg_arr).reshape(-1,300)
neg_la=np.array(neg_la).reshape((-1,1))

In [ ]:
data_pos=np.concatenate((pos_arr,pos_la),axis=1)
data_neg=np.concatenate((neg_arr,neg_la),axis=1)
data_pos=data_pos[~np.isnan(data_pos).any(axis=1)]
data_neg=data_neg[~np.isnan(data_neg).any(axis=1)]

In [ ]:
dataX=np.concatenate((data_pos[:,:-1],data_neg[:,:-1]),axis=0)
dataY=np.concatenate((data_pos[:,-1],data_neg[:,-1]),axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(dataX,dataY,test_size=0.1)

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
lr_models=[]
for s in ['liblinear','lbfgs','newton-cg','sag']:
    for m in [1000,3000,5000]:    #这三种迭代次数的结果是一样的
        lr=LogisticRegression(max_iter=m,solver=s) 
        lr.fit(train_x,train_y)
        lr_model_name=s+'_'+str(m)+'_'+'logistic'+'.pickle'
        lr_models.append(lr_model_name)
        with open(lr_model_name, 'wb') as fw:
            pickle.dump(lr, fw)

for m in lr_models:
    with open(m, 'rb') as fr:
        new_lr = pickle.load(fr)
        print (m+str(new_lr.score(test_x,test_y)))

#liblinear_1000_logistic.pickle0.8615847542627884  #与svc相比速度快很多
# liblinear_1000_logistic.pickle0.8615847542627884
# liblinear_3000_logistic.pickle0.8615847542627884
# liblinear_5000_logistic.pickle0.8615847542627884
# lbfgs_1000_logistic.pickle0.8615011701771983
# lbfgs_3000_logistic.pickle0.8615011701771983
# lbfgs_5000_logistic.pickle0.8615011701771983
# newton-cg_1000_logistic.pickle0.8615011701771983
# newton-cg_3000_logistic.pickle0.8615011701771983
# newton-cg_5000_logistic.pickle0.8615011701771983
# sag_1000_logistic.pickle0.8615011701771983
# sag_3000_logistic.pickle0.8615011701771983
# sag_5000_logistic.pickle0.8615011701771983

In [ ]:
import pickle
from sklearn import neural_network
nn_models=[]
for s in ['lbfgs']:
    for m in [3000]:    #这三种迭代次数的结果是一样的
        for hls in [(300,50),(200,100)]:
            nn=neural_network.MLPClassifier(hidden_layer_sizes=hls,solver=s,max_iter=m,early_stopping=True)
            nn.fit(train_x,train_y)
            nn_model_name=s+'_'+str(m)+'_'+str(hls)+'network'+'.pickle'
            nn_models.append(nn_model_name)
            with open(nn_model_name, 'wb') as fw:
                pickle.dump(nn, fw)
#lbfgs_3000_(200, 50)network.pickle0.9762621196924106

nn_models=['adam_3000_(100, 50)network.pickle','adam_3000_(100,)network.pickle',
'adam_3000_(200, 50)network.pickle',
'lbfgs_3000_(100,)network.pickle',
'lbfgs_3000_(200, 50)network.pickle',
'lbfgs_3000_(300, 50)network.pickle',
'lbfgs_3000_(200, 100)network.pickle',
'sgd_3000_(100,)network.pickle',
'sgd_3000_(200, 50)network.pickle']
for m in nn_models:
    with open(m, 'rb') as fr:
        new_nn = pickle.load(fr)
        print (m+str(new_nn.score(test_x,test_y)))

# adam_3000_(100, 50)network.pickle0.9164994984954865
# adam_3000_(100,)network.pickle0.9127382146439318
# adam_3000_(200, 50)network.pickle0.9132397191574724
# lbfgs_3000_(100,)network.pickle0.9758441992644601
# lbfgs_3000_(200, 50)network.pickle0.9784353059177533
# lbfgs_3000_(300, 50)network.pickle0.8593279839518556
# lbfgs_3000_(200, 100)network.pickle0.8567368772985624
# sgd_3000_(100,)network.pickle0.8849047141424273
# sgd_3000_(200, 50)network.pickle0.9035439652290204

In [ ]:
import pickle
from sklearn.decomposition import PCA
pca = PCA(n_components=64)  
pca_fitted=pca.fit(train_x)
with open('pca_demo64.pickle','wb') as fw:
    pickle.dump(pca_fitted, fw)
train_x=pca_fitted.transform(train_x)
# print(pca.explained_variance_ratio_)
test_x=pca_fitted.transform(test_x)

In [ ]:
import pickle
from sklearn.svm import SVC
#设计不同参数的分类器并保存模型
models=[]
for k in ['linear','poly','rbf','sigmoid']:
    for c in [0.1,1,2,8]:
        clf = SVC(kernel=k,C=c)
        clf.fit(train_x,train_y)  
        model_name=k+'_'+str(c)+'_'+'svm'+'.pickle'
        models.append(model_name)
        with open(model_name, 'wb') as fw:
            pickle.dump(clf, fw)


for m in models:
    with open(m, 'rb') as fr:
        new_svm = pickle.load(fr)
        print (m+str(new_svm.score(test_x,test_y)))
#rbf_2_svm.pickle0.8753761283851554   #速度贼慢...
# linear_0.1_svm.pickle0.8525576730190572
# linear_1_svm.pickle0.8524740889334671
# linear_2_svm.pickle0.8526412571046472
# linear_8_svm.pickle0.8525576730190572
# poly_0.1_svm.pickle0.8445336008024072
# poly_1_svm.pickle0.8585757271815446
# poly_2_svm.pickle0.859662320294216
# poly_8_svm.pickle0.861250417920428
# rbf_0.1_svm.pickle0.8658475426278837
# rbf_1_svm.pickle0.8750417920427951
# rbf_2_svm.pickle0.8753761283851554
# rbf_8_svm.pickle0.8742895352724841
# sigmoid_0.1_svm.pickle0.7433968572383818
# sigmoid_1_svm.pickle0.7610330992978936
# sigmoid_2_svm.pickle0.7608659311267135
# sigmoid_8_svm.pickle0.7607823470411234

In [ ]:
def sentence_vec(sentence):
    import jieba
    # from gensim.models.word2vec import Word2Vec
    # model = Word2Vec.load('wiki_w2v.model')
    senvec=np.zeros((1,300))
    newsen=jieba.cut(sentence)
    length=0
    for i in newsen:
        try:
            senvec+=model[i].reshape((1,300))
            length+=1
        except KeyError:
            continue 
    return senvec/length

In [ ]:
def data_label_split(sourse):
    label=[]
    data=[]    
    for line in sourse.values:
        label.append(float(line[0]))
        data.append(sentence_vec(line[1]))
    return np.array(data).reshape((-1,300)),np.array(label)

In [ ]:
datas,labels=data_label_split(news)
datas=datas[~np.isnan(datas).any(axis=1)]
labels=np.ones(datas.shape[0],)

In [ ]:
bad=pd.read_csv('bad.csv',names=['price','describe'],sep='\t')
bad=bad.drop(index=0)
bad_datas,bad_labels=data_label_split(bad)
bad_datas=bad_datas[~np.isnan(bad_datas).any(axis=1)]
bad_labels=np.zeros(bad_datas.shape[0],)

In [ ]:
bad2=pd.read_csv('bad2.csv',names=['price','describe'],sep='\t')
bad2=bad2.drop(index=0)
bad2_datas,bad2_labels=data_label_split(bad2)
bad2_datas=bad2_datas[~np.isnan(bad2_datas).any(axis=1)]
bad2_labels=np.zeros(bad2_datas.shape[0],)

In [ ]:
badX=np.concatenate((bad2_datas,bad_datas),axis=0)
badY=np.concatenate((bad2_labels,bad_labels),axis=0)

In [ ]:
newtestX=np.concatenate((datas,badX),axis=0)
newtestY=np.concatenate((labels,badY),axis=0)

In [ ]:
import pickle
for models in [
    'adam_3000_(100, 50)network.pickle',
    'adam_3000_(100,)network.pickle',
    'adam_3000_(200, 50)network.pickle',
    'lbfgs_3000_(100,)network.pickle',
    'lbfgs_3000_(200, 50)network.pickle',
    'lbfgs_3000_(200, 100)network.pickle',
    'lbfgs_3000_(300, 50)network.pickle',
    'sgd_3000_(200, 50)network.pickle',
    'sgd_3000_(100,)network.pickle',
    'lbfgs_1000_logistic.pickle',
    'liblinear_1000_logistic.pickle',
    'newton-cg_3000_logistic.pickle',
    'sag_1000_logistic.pickle'
#     'test_rbf_2_svm.pickle'  #需要降维
]:
    with open(models, 'rb') as fr:
        new_nn = pickle.load(fr)
        print(models +str(new_nn.score(datas,labels)))

#模型在好评上的表现
# adam_3000_(100, 50)network.pickle0.8759842519685039
# adam_3000_(100,)network.pickle0.8523622047244095
# adam_3000_(200, 50)network.pickle0.8228346456692913
# lbfgs_3000_(100,)network.pickle0.7598425196850394
# lbfgs_3000_(200, 50)network.pickle0.7716535433070866
# lbfgs_3000_(200, 100)network.pickle0.7736220472440944
# lbfgs_3000_(300, 50)network.pickle0.7460629921259843
# sgd_3000_(200, 50)network.pickle0.8405511811023622
# sgd_3000_(100,)network.pickle0.7992125984251969
# lbfgs_1000_logistic.pickle0.7795275590551181
# liblinear_1000_logistic.pickle0.7775590551181102
# newton-cg_3000_logistic.pickle0.7775590551181102
# sag_1000_logistic.pickle0.7775590551181102

In [ ]:
import pickle
for models in [
    'adam_3000_(100, 50)network.pickle',
    'adam_3000_(100,)network.pickle',
    'adam_3000_(200, 50)network.pickle',
    'lbfgs_3000_(100,)network.pickle',
    'lbfgs_3000_(200, 50)network.pickle',
    'lbfgs_3000_(200, 100)network.pickle',
    'lbfgs_3000_(300, 50)network.pickle',
    'sgd_3000_(200, 50)network.pickle',
    'sgd_3000_(100,)network.pickle',
    'lbfgs_1000_logistic.pickle',
    'liblinear_1000_logistic.pickle',
    'newton-cg_3000_logistic.pickle',
    'sag_1000_logistic.pickle'
#     'test_rbf_2_svm.pickle'  #需要降维
]:
    with open(models, 'rb') as fr:
        new_nn = pickle.load(fr)
        print(models +str(new_nn.score(badX,badY)))
        
#模型在差评上的表现
# adam_3000_(100, 50)network.pickle0.6831325301204819
# adam_3000_(100,)network.pickle0.744578313253012
# adam_3000_(200, 50)network.pickle0.7518072289156627
# lbfgs_3000_(100,)network.pickle0.6771084337349398
# lbfgs_3000_(200, 50)network.pickle0.736144578313253
# lbfgs_3000_(200, 100)network.pickle0.7240963855421687
# lbfgs_3000_(300, 50)network.pickle0.7108433734939759
# sgd_3000_(200, 50)network.pickle0.7048192771084337
# sgd_3000_(100,)network.pickle0.7240963855421687
# lbfgs_1000_logistic.pickle0.7518072289156627
# liblinear_1000_logistic.pickle0.7518072289156627
# newton-cg_3000_logistic.pickle0.7518072289156627
# sag_1000_logistic.pickle0.7518072289156627

In [ ]:
import pickle
with open('pca_demo64.pickle','rb') as fr:
    pca_demo=pickle.load(fr)
    data_pcaed=pca_demo.transform(newtestX)
    print(data_pcaed.shape)
for m in [
'linear_0.1_svm.pickle',
'linear_1_svm.pickle',
'linear_2_svm.pickle',
'linear_8_svm.pickle',
'poly_0.1_svm.pickle',
'poly_1_svm.pickle',
'poly_2_svm.pickle',
'poly_8_svm.pickle',
'rbf_0.1_svm.pickle',
'rbf_1_svm.pickle',
'rbf_2_svm.pickle',
'rbf_8_svm.pickle',
'sigmoid_0.1_svm.pickle',
'sigmoid_1_svm.pickle',
'sigmoid_2_svm.pickle',
'sigmoid_8_svm.pickle',
]:
    with open(m, 'rb') as fr:
        new_model= pickle.load(fr)
        print(m +str(new_model.score(data_pcaed,newtestY)))

#差评多数、好评少数时的表现：
#poly_0.1_svm.pickle0.7675635276532138
# rbf_0.1_svm.pickle0.7630792227204783    
# rbf_2_svm.pickle0.7615844544095666

# linear_0.1_svm.pickle0.7511210762331838
# linear_1_svm.pickle0.7511210762331838
# linear_2_svm.pickle0.7511210762331838
# linear_8_svm.pickle0.7511210762331838
# poly_0.1_svm.pickle0.7675635276532138
# poly_1_svm.pickle0.750373692077728
# poly_2_svm.pickle0.7466367713004485
# poly_8_svm.pickle0.7414050822122571
# rbf_0.1_svm.pickle0.7630792227204783
# rbf_1_svm.pickle0.7593423019431988
# rbf_2_svm.pickle0.7615844544095666
# rbf_8_svm.pickle0.7316890881913304
# sigmoid_0.1_svm.pickle0.6741405082212257
# sigmoid_1_svm.pickle0.6935724962630793
# sigmoid_2_svm.pickle0.6928251121076233
# sigmoid_8_svm.pickle0.6935724962630793

#好评结果   # rbf_0.1_svm.pickle0.8385826771653543
# linear_0.1_svm.pickle0.8011811023622047
# linear_1_svm.pickle0.8011811023622047
# linear_2_svm.pickle0.8011811023622047
# linear_8_svm.pickle0.8011811023622047
# poly_0.1_svm.pickle0.7480314960629921
# poly_1_svm.pickle0.7539370078740157
# poly_2_svm.pickle0.75
# poly_8_svm.pickle0.7696850393700787
# rbf_0.1_svm.pickle0.8385826771653543
# rbf_1_svm.pickle0.8208661417322834
# rbf_2_svm.pickle0.8149606299212598
# rbf_8_svm.pickle0.7854330708661418
# sigmoid_0.1_svm.pickle0.5728346456692913
# sigmoid_1_svm.pickle0.6633858267716536
# sigmoid_2_svm.pickle0.6633858267716536
# sigmoid_8_svm.pickle0.6633858267716536


#差评上   # poly_0.1_svm.pickle0.8200589970501475  # poly_0.1_svm.pickle0.7795180722891566
# linear_0.1_svm.pickle0.7204819277108434
# linear_1_svm.pickle0.7204819277108434
# linear_2_svm.pickle0.7204819277108434
# linear_8_svm.pickle0.7204819277108434
# poly_0.1_svm.pickle0.7795180722891566
# poly_1_svm.pickle0.7481927710843373
# poly_2_svm.pickle0.744578313253012
# poly_8_svm.pickle0.7240963855421687
# rbf_0.1_svm.pickle0.7168674698795181
# rbf_1_svm.pickle0.7216867469879518
# rbf_2_svm.pickle0.7289156626506024
# rbf_8_svm.pickle0.6987951807228916
# sigmoid_0.1_svm.pickle0.736144578313253
# sigmoid_1_svm.pickle0.7120481927710843
# sigmoid_2_svm.pickle0.7108433734939759
# sigmoid_8_svm.pickle0.7120481927710843

In [ ]:
#模型在新的好评与差评上的表现（差评居多）
# adam_3000_(100, 50)network.pickle0.7563527653213752
# adam_3000_(100,)network.pickle0.7855007473841554
# adam_3000_(200, 50)network.pickle0.7787742899850523
#poly_0.1_svm.pickle0.7675635276532138
# rbf_0.1_svm.pickle0.7630792227204783    
# rbf_2_svm.pickle0.7615844544095666
# （lbfgs_1000_logistic.pickle0.7623318385650224
# liblinear_1000_logistic.pickle0.7615844544095666
# newton-cg_3000_logistic.pickle0.7615844544095666
# sag_1000_logistic.pickle0.7615844544095666）

#差评表现：
# poly_0.1_svm.pickle0.7795180722891566
# adam_3000_(100,)network.pickle0.744578313253012
# adam_3000_(200, 50)network.pickle0.7518072289156627
# （lbfgs_1000_logistic.pickle0.7518072289156627
# liblinear_1000_logistic.pickle0.7518072289156627
# newton-cg_3000_logistic.pickle0.7518072289156627
# sag_1000_logistic.pickle0.7518072289156627）

#好评表现：
# adam_3000_(100, 50)network.pickle0.8759842519685039
# adam_3000_(100,)network.pickle0.8523622047244095
# adam_3000_(200, 50)network.pickle0.8228346456692913
# rbf_0.1_svm.pickle0.8385826771653543
# rbf_1_svm.pickle0.8208661417322834
# （lbfgs_1000_logistic.pickle0.7795275590551181
# liblinear_1000_logistic.pickle0.7775590551181102
# newton-cg_3000_logistic.pickle0.7775590551181102
# sag_1000_logistic.pickle0.7775590551181102）